# CrewAI

How to use:
- Run "Setup"
- Jump to one of the chapters and execute the cells, all chapters are independent from each other

# TOC

1. [CrewAI: Tool Tests](#CrewAI:-Tool-Tests)
1. [Web Researcher](#1.-Web-Researcher)

# Notes

## Tools

- ServerDevTool (Google) works better than DuckDuckGoSearchRun (DuckDuckGo). Eg. it provides the dates of the found articles, which DuckDuckGoSearchRun does not do

# Setup

In [1]:
#!uv pip install jupyterlab==4.2.1 pyautogen==0.2.27 crewai==0.30.11 crewai-tools==0.2.6 duckduckgo-search==6.1.4 agentops==0.2.0

In [2]:
#!uv pip freeze > requirements.txt
#uv pip install -r requirements.txt

In [9]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [10]:
import json
import os
from getpass import getpass
import psutil
import openai
IN_NOTEBOOK = any(["jupyter-lab" in i for i in psutil.Process().parent().cmdline()])
if IN_NOTEBOOK:
  CREDS = json.loads(getpass("Secrets (JSON string): "))
  os.environ['CREDS'] = json.dumps(CREDS)
  CREDS = json.loads(os.getenv('CREDS'))

In [11]:
from os import environ
if environ.get('OPENAI_API_KEY') is None:
    print('Environment variable not found. Setting values...')
    os.environ["OPENAI_API_KEY"] = CREDS['OpenAI']['v2']['credential'] 
    os.environ["TOGETHERAI_API_KEY"] = CREDS['together-ai']['key']['credential']
    os.environ['ANTHROPIC_API_KEY'] = CREDS['anthropic']['key']['credential']
    os.environ["SERPER_API_KEY"] = CREDS['serper_dev']['key']['credential']
    #openai.api_key = CREDS['OpenAI']['v2']['credential'] 

In [12]:
# Enable Langchain Tracing
#os.environ["LANGCHAIN_API_KEY"] = CREDS['langsmith']['key']['credential']
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [13]:
# Enable Agentop Tracing
#os.environ["AGENTOPS_API_KEY"] = CREDS['agentops']['key']['credential']
#import agentops
#agentops.init()

# Get list of available OpenAI models

In [14]:
client = openai.OpenAI()
model_list = client.models.list()
for model in model_list:
  print(model.id)

dall-e-3
gpt-4-1106-preview
whisper-1
davinci-002
babbage-002
dall-e-2
gpt-3.5-turbo-16k
tts-1-hd-1106
tts-1-hd
gpt-3.5-turbo-1106
gpt-4o-2024-05-13
gpt-4o
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
text-embedding-3-small
gpt-4-turbo-2024-04-09
tts-1
gpt-4-turbo
gpt-4-0125-preview
gpt-3.5-turbo-0125
gpt-3.5-turbo
gpt-3.5-turbo-0301
gpt-4-turbo-preview
tts-1-1106
text-embedding-3-large
gpt-3.5-turbo-0613
gpt-4
text-embedding-ada-002
gpt-4-1106-vision-preview
gpt-3.5-turbo-16k-0613
gpt-4-0613
gpt-4-vision-preview


# CrewAI: Tool Tests

In [15]:
from crewai import Agent
from crewai_tools import SerperDevTool
from langchain_community.tools import DuckDuckGoSearchRun

os.environ["OPENAI_MODEL_NAME"]="gpt-4o" # or "gpt-4-1106-preview"

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Answer the following question: {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "information research, eager to explore and find knowledge."
  ),
  #tools=[search_tool],
  #allow_delegation=True
)

from crewai import Task


# Writing task with language model configuration
answer = Task(
  description=(
    "Answer the question '{topic}'."
    "Answer in a short and precise way, do not add anything else." # Use bulletpoints to structure your answer. "
    "Write your answer to a file."
  ),
  expected_output="The answer to the question '{topic}'", #,formatted as markdown.
  #tools=[search_tool_google],
  agent=researcher,
  async_execution=False,
  output_file='agent-output.txt'  # This is necessary to create the output as file
)

from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher],
  tasks=[answer],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=False,
  max_rpm=100,
  share_crew=True
)

2024-06-03 05:03:29,551 - 140656029559680 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [16]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'What is 10+20?'}) 
print(result)



> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer: 10 + 20 equals 30

> Finished chain.
10 + 20 equals 30


In [10]:
from IPython.display import Markdown,display
display(Markdown(result))

My best complete final answer to the task is 10 + 20 = 30.

# 1. Web Researcher 

- searches Google and returns Top 5 results report
- adopted from example below

In [20]:
from crewai import Agent
from crewai_tools import SerperDevTool
from langchain_community.tools import DuckDuckGoSearchRun

search_tool_google = SerperDevTool()
search_tool_duckduckgo = DuckDuckGoSearchRun()

search_tool = search_tool_google

os.environ["OPENAI_MODEL_NAME"]="gpt-3.5-turbo" # or "gpt-4o" # or "gpt-4-1106-preview"

# Creating a senior researcher agent
researcher = Agent(
  role='Senior Researcher',
  goal='Find most relevant sources in the web {topic}',
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "information research, eager to explore and find knowledge."
  ),
  tools=[search_tool],
  allow_delegation=False,
	verbose=True
)

# Creating a writer agent with 
writer = Agent(
  role='Writer',
  goal='Writing precise reports about {topic}',
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "reports and summaries on an expert level."
  ),
  allow_delegation=False,
  verbose=True
)

In [21]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Search the web for the most relevant articles about {topic} in 2024."
    "Focus on sources like scientific papers, newspapers and expert blogs."
    "Your final report should list the 5 most relevant articles including URL, date, headline and a short summary."
  ),
  expected_output='A comprehensive list of search results.',
  tools=[search_tool],
  agent=researcher
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful report on {topic}."
    "Take the given articles and write for each of them one chapter. Include the source URL and date."
  ),
  expected_output='A 4 report on {topic}, listing the most relevant sources, formatted as markdown.',
  #tools=[search_tool_google],
  agent=writer
)

In [22]:
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

2024-06-03 05:11:16,622 - 140656029559680 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [23]:
# Starting the task execution process with enhanced feedback
# result = crew.kickoff(inputs={'topic': 'will donald trumps chances of winning the presidential election diminish after his conviction in court?'})
# result = crew.kickoff(inputs={'topic': 'Will Apples Siri get smarter in 2024?'}) 
result = crew.kickoff(inputs={'topic': 'Will Apples Siri get smarter in 2024? Answer in German language'}) 
print(result)



> Entering new CrewAgentExecutor chain...
I need to carefully search the web for the most relevant articles about whether Apple's Siri will get smarter in 2024. I should focus on scientific papers, newspapers, and expert blogs to find the most reliable information.

Action: Search the internet
Action Input: {"search_query": "Wird Apples Siri 2024 intelligenter?"} 


Search results: Title: Better Siri is coming: what Apple's research says about its AI plans
Link: https://www.theverge.com/2024/5/5/24147995/apple-siri-ai-research-chatbot-creativity
Snippet: For the last few years, Apple has been looking into ways to use AI to improve Siri, give tools to artists, improve health data, and more.
---
Title: Will Siri ever become more intelligent? : r/apple - Reddit
Link: https://www.reddit.com/r/apple/comments/10x2b95/will_siri_ever_become_more_intelligent/
Snippet: They know it. Apple will release siri 2.0 in 2023 or 2024.
---
Title: Apple's AI-based Siri assistant to let you control app f

In [24]:
from IPython.display import Markdown,display
display(Markdown(result))

# Report: Wird Apples Siri 2024 intelligenter?

## Einleitung
Die Zukunft von Apples Siri wirft viele Fragen auf, insbesondere in Bezug auf ihre Intelligenz und Funktionalität im Jahr 2024. Verschiedene Quellen deuten darauf hin, dass Apple intensiv an der Weiterentwicklung von Siri arbeitet, um sie leistungsfähiger und vielseitiger zu machen. In diesem Bericht werden die neuesten Erkenntnisse und Prognosen aus verschiedenen Artikeln zu diesem Thema zusammengefasst.

### 1. Better Siri is coming: what Apple's research says about its AI plans
- Quelle: [The Verge](https://www.theverge.com/2024/5/5/24147995/apple-siri-ai-research-chatbot-creativity)
- Datum: 5. Mai 2024
- Zusammenfassung: Apple untersucht Wege, wie KI eingesetzt werden kann, um Siri zu verbessern, Künstlern Werkzeuge zur Verfügung zu stellen, Gesundheitsdaten zu verbessern und mehr.

### 2. Apple's AI-based Siri assistant to let you control app functions with ...
- Quelle: [Business Standard](https://www.business-standard.com/technology/tech-news/apple-s-ai-based-siri-assistant-to-let-you-control-app-functions-with-voice-124053100518_1.html)
- Datum: 31. Mai 2024
- Zusammenfassung: Das neue System wird es dem Siri-Assistenten ermöglichen, ein iPhone oder iPad präziser zu steuern und zu navigieren.

### 3. Apple may launch a new gen AI-powered version of Siri at WWDC ...
- Quelle: [Times of India](https://timesofindia.indiatimes.com/gadgets-news/apple-may-launch-a-new-gen-ai-powered-version-of-siri-at-wwdc-2024-what-to-expect/articleshow/106583931.cms)
- Datum: Nicht angegeben
- Zusammenfassung: Es wird gemunkelt, dass Apple auf der WWDC 2024 ein großes Upgrade für Siri vorstellen wird. Die neue Version von Siri wird von fortschrittlicher künstlicher Intelligenz angetrieben.

### 4. Prediction: The Vision Pro isn't Apple's ace in the hole for 2024; Siri ...
- Quelle: [Phone Arena](https://www.phonearena.com/news/prediction-vision-pro-isnt-apple-ace-in-the-hole-for-2024-siri-about-to-evolve-big-time-with-generative-ai_id156021)
- Datum: Nicht angegeben
- Zusammenfassung: Das neue Siri 2024 könnte über ChatGPT-ähnliche Konversationsfähigkeiten verfügen; tatsächliches Verständnis von geschriebenem und gesprochenem Wort, nicht nur grundlegende Befehle.

### 5. Siri AI updates at WWDC 2024: What is Apple planning for its virtual ...
- Quelle: [India Today](https://www.indiatoday.in/amp/technology/news/story/siri-ai-update-wwdc-what-is-apple-planning-for-its-virtual-assistant-2546057-2024-05-31)
- Datum: 31. Mai 2024
- Zusammenfassung: Apple plant, Siris Software zu überarbeiten, um sie mit Hilfe von KI fortschrittlicher zu machen. Mit dem neuen Update wird Siri über fortschrittliche Funktionen verfügen.

Durch die Zusammenfassung und Analyse dieser Artikel lässt sich vermuten, dass Apples Siri im Jahr 2024 voraussichtlich intelligenter und leistungsfähiger sein wird. Mit dem Einsatz von fortgeschrittener künstlicher Intelligenz und innovativen Funktionen könnte Siri zu einem noch nützlicheren virtuellen Assistenten werden. Es bleibt spannend, die zukünftigen Entwicklungen von Siri im Auge zu behalten.

# 2. Blog post writer: Researcher, Writer

In [5]:
from crewai import Agent
from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

os.environ["OPENAI_MODEL_NAME"]="gpt-4o" # or "gpt-4-1106-preview"

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

In [6]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points,"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

In [7]:
from crewai import Crew, Process

# Forming the tech-focused crew with some enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True
)

In [ ]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'}) # in healthcare # in software development
print(result)

In [9]:
print(result)

```markdown
# AI in Healthcare: A 2023 Snapshot of Innovation and Impact

## The Pulse of Progress: Analytics, Diagnostics, Robotics, and Telehealth
The healthcare landscape is witnessing a thrilling upsurge in innovation, thanks to Artificial Intelligence (AI). In 2023, AI continues to redefine healthcare analytics, diagnostics, robotics, and telehealth. Data is no longer dormant as AI breathes life into numbers, translating them into actionable insights. Diagnostics are swifter and more precise, with AI algorithms interpreting complex medical images to detect diseases that may slip past human scrutiny. Surgical robots, enhanced by AI, are becoming the norm rather than the exception, enabling surgeons to perform intricate procedures with remarkable precision. Additionally, telehealth has evolved to offer more personalized care through AI, ensuring healthcare is accessible to all, irrespective of geographical barriers.

## Real-World Impact: Case Studies and Examples
Real-world applica

In [10]:
print(result)

```markdown
# AI: The New Power Tool in Software Development

In the bustling world of software development, 2023 has been a pivotal year of transformation, with machine learning and data science becoming core pillars of innovation. According to recent industry reports, there's a surge in roles tailored to ML, signaling a shift towards intelligent automation and informed decision-making. The integration of AI is not just a trend; it's a revolutionary approach that refines how we build the future of technology. Imagine developers, equipped with predictive analytics, crafting applications that not only meet user needs but anticipate them.

The modern developer's toolkit is brimming with AI-driven marvels. Tools like GitHub Copilot, powered by OpenAI's Codex, and debugging wizards like Stepsize are redefining the craft of coding. They automate the mundane, offer intelligent code suggestions, and free up the creative mind to tackle the grand challenges of software engineering. AI-infused c

# 3. Blog post writer: Planner, Writer, Editor

Blog post writer Multi-Agent System using Crewai and Ollama

Source: https://medium.com/the-ai-forum/create-a-blog-writer-multi-agent-system-using-crewai-and-ollama-f47654a5e1cd

In [12]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model = "crewai-llama3",
    base_url = "http://localhost:11434/v1")

In [13]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic} in 'https://medium.com/'."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "You have to prepare a detailed "
              "outline and the relevant topics and sub-topics that has to be a part of the"
              "blogpost."
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    #llm=llm,
    allow_delegation=False,
 verbose=True
)
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic} in 'https://medium.com/'. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    #llm=llm,
    verbose=True
)
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization 'https://medium.com/'. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    #llm=llm,
    allow_delegation=False,
    verbose=True
)

In [14]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
  "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)
inputs = {"topic":"Comparative study of LangGraph, Autogen and Crewai for building multi-agent system."}
result = crew.kickoff(inputs=inputs)

In [19]:
from IPython.display import Markdown,display
display(Markdown(result))

```markdown
# A Comparative Analysis of MAS Frameworks: LangGraph, Autogen, and Crewai

## Exploring Multi-Agent Systems in AI Development

Multi-agent systems (MAS) stand at the forefront of artificial intelligence, driving the creation of sophisticated, intelligent, and collaborative AI solutions. This technology enables multiple autonomous entities to interact, learn, and make decisions collectively, facilitating progress in various fields such as robotics and urban planning. This article presents a detailed comparison of three notable MAS frameworks: LangGraph, Autogen, and Crewai, providing valuable perspectives for software developers, AI researchers, and project managers involved in AI-centric initiatives.

## Deciphering the MAS Frameworks Terrain

Imagine an orchestra where each musician—the agent—plays a part in the harmony of the collective symphony—the system. Originating from the realm of distributed AI, MAS frameworks have expanded their capabilities to tackle intricate, real-world challenges through the synergy of collaborating agents. The question is, how can one choose the most suitable framework for their project's needs?

## LangGraph: The Architect of Structure

LangGraph stands out in the MAS domain with its penchant for structured environments, where it thrives by interpreting complex relationship dynamics and data models. It is particularly effective in powering systems such as recommendation engines and intricate social network analytics.

**Strengths:**
- Mastery in explicit relationship modeling.
- Proficient in managing structured data sets.

**Drawbacks:**
- Limited effectiveness in unstructured environments.

**Community Presence and Resources:**
- An expanding user base with a penchant for structured solutions.
- Thorough documentation catering to a range of established use cases.

## Autogen: The Dynamic Trailblazer

Autogen is lauded for its flexible and adaptive nature, positioning it as the framework of choice for sectors characterized by volatility and unpredictability.

**Strengths:**
- Remarkable adaptability to ever-changing environments.
- Enables spontaneous creation of agents as needed.

**Drawbacks:**
- Complexity may steepen the initial learning curve.

**Community Engagement and Support:**
- A dynamic forum fostering active user participation.
- A wealth of educational materials facilitating user proficiency.

## Crewai: The Cooperative Strategist

Crewai sets itself apart through its specialized focus on collaborative endeavors, making it particularly adept for projects that demand intricate agent cooperation, such as the coordination of autonomous vehicles.

**Advantages:**
- Exceptional capabilities for agent teamwork.
- Intuitive framework design for managing complex tasks.

**Disadvantages:**
- Higher resource demand for peak performance.

**Resource Availability and Guidance:**
- A dedicated support system that's easily accessible.
- Comprehensive documentation tailored to collaborative ventures.

## Comparative Features at a Glance

When considering LangGraph, Autogen, and Crewai side by side, important aspects such as scalability, compatibility with existing infrastructures, and user-friendliness become critical decision factors.

- LangGraph is the front-runner in structured data processing.
- Autogen stands out for its swift adaptability to shifts in requirements.
- Crewai excels at nurturing inter-agent cooperation.

Insights from industry benchmarks and practical case studies shine a light on their effectiveness in scenarios like traffic optimization and economic forecasting.

## Identifying Trends and Preparing for What's Next

Staying updated with the latest market trends is imperative. Langraph, Autogen, and Crewai are continuously refined, with enhancements that mirror cutting-edge AI research. As MAS applications expand, so does the sophistication of these frameworks, signaling an era of more seamless and independent AI operations.

## Selecting the Ideal MAS Framework for Your AI Project

Choosing the right MAS framework is a crucial step that involves evaluating project dimensions, agent intricacy, and sustainability. Utilizing a comparison chart or decision matrix can be an invaluable strategy, providing a solid foundation for a well-informed choice.

## Engage With the Community

We invite you to share your personal experiences and knowledge gained from working with any of these MAS frameworks in the comments section. Join the ongoing dialogue with like-minded AI aficionados and experts. For further information and to join the wave of innovation, follow the links provided and engage with the broader community seeking smarter, interconnected AI solutions.

## Final Thoughts

In summation, LangGraph, Autogen, and Crewai each bring distinct advantages to the table, catering to various facets of multi-agent system development. The optimal framework for a given project will depend on the unique requirements at hand and the overarching goals of the AI system. As we witness the continuous evolution of MAS frameworks, staying informed and versatile is the key to unlocking their vast potential.

The future of AI is one of collaboration, adaptability, and boundless possibilities—let the framework you select be a cornerstone in building that future.
```

# 4. Poem Writer: Manager, Researcher, Writer

Source: https://medium.com/thedeephub/crafting-custom-love-poems-with-crewai-agents-5ee3051b4414

In [7]:
from crewai import Agent, Task, Crew, Process
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import load_tools
from langchain.tools import tool
from crewai.tasks.task_output import TaskOutput
from langchain_openai import ChatOpenAI

In [10]:
# Define the topic 
# In this case, since we want a poem for our loved one, the topic should be things that they like and that you want to include in the poem
topic = "Taylor Swift and long walks on the beach at sunset"

In [11]:
# Define the search tool 
search_tool = DuckDuckGoSearchRun()

# Function that is run after the task is completed
def callback_function(output: TaskOutput):
    ## What to do after the task is completed (i.e. send an sms with a love poem)
    print(f"""
        Task completed with the following output:
        Task: {output.description}
        Output: {output.result}
    """)

# Custom tool - read webpage
class ContentTools:
    @tool("Read webpage content")
    def read_content(url: str) -> str:
        """Read the content of a webpage"""
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        text_content = soup.get_text()
        return text_content[:5000]

In [12]:
# First agent: the love poem manager
manager = Agent(
    role = "Love Poem Manager",
    goal = "Manage the process of creating a love poem for the partner of the client",
    verbose = True,
    backstory = "With a mind for romance, I want to deliver the best custom love poem to my client to give to their partner for Valentines Day",
    allow_delegation = True,
    max_iter = 10,
    max_rpm = 20,
)
# Second agent: the love poem searcher
researcher = Agent(
    role = "Love Poem Searcher",
    goal = f"Find a love poem for my partner, who really likes {topic}",
    verbose = True,
    backstory = """You are a curious love poem searcher, and you want to 
    find the best and most custom love poem for your client to give to their partner, 
    therefore you need some details about the things that the partner really likes! You will search for these details.""",
)
# Third agent: the love poem writer
writer = Agent(
    role = "Love Poem Writer",
    goal = f"Write a love poem for my partner, who really likes {topic}",
    verbose = True,
    backstory = """You are a creative love poem writer, and you want to 
    write the best and most custom love poem for your client to give to their partner. 
    With a mind for romance and a deep love for poetry, you find the perfect poetry style 
    for your client and create a wonderful poem that conveys love, respect, kindness, and admiration""",
)

In [13]:
# Task 1: List ideas for a love poem
list_ideas = Task(
    description = "List of 5 cute ideas for a love poem, including possible poetry styles",
    expected_output = "Bullet point of 5 cute ideas for a love poem",
    tools=[search_tool, ContentTools().read_content],
    agent = researcher,
    async_execution=True,
)

# Task 2: List important facts related to topic
list_facts = Task(
    description = f"List of 5 important facts specifically for someone that likes {topic}",
    expected_output = "Bullet point of 5 important facts related to the given topic for a love poem",
    tools=[search_tool, ContentTools().read_content],
    agent = researcher,
    async_execution=True,
)

# Task 3: Write the love poem
write_poem = Task(
    description = f"Write a poem that would make Romeo and Juliet blush. The poem should be custom and tailored to the client's partner, who really likes {topic}",
    expected_output = "A lovely poem in a style that the client would like",
    tools=[search_tool],
    agent = writer,
    context=[list_ideas, list_facts],
    callback=callback_function,
)

# Task 4: Coordinate the love poem
manager_task = Task(
    description=f"""Oversee the creation of a great love poem for the client, 
    a lovebird who's looking for a custom poem for their partner, who really likes {topic}. 
    Ensure that the poem is custom and tailored to the client's partner and in a style that 
    is appropriate and conveys love, respect, kindness, and admiration""",
    expected_output=f"A lovely poem in a style that the client would like that incoporates the ideas and facts related to the {topic}",
    agent=manager,
)

In [ ]:
crew = Crew(
    agents = [manager, researcher, writer],
    tasks = [list_ideas, list_facts, write_poem, manager_task],
    process = Process.hierarchical,
    manager_llm=ChatOpenAI(temperature=0, model="gpt-4"),
)
result = crew.kickoff()

In [15]:
print(result)

'Sonnet to the Swift Sunset' (Shakespearean Sonnet/Free Verse)

In the glow of the setting sun, our hands entwined,
Upon the sandy shore, where the ocean's chorus is intertwined.
The melody of your laughter, as enchanting as a Swift's song,
Your eyes, a mirror of the sea, where my love for you is strong.

Each footprint we leave, a testament of our story,
Under the canvas of the sunset, painted with Swift's lyrical glory.
Our hearts beat in rhythm, to the sea's lullaby, and Swift's harmonious tune,
As we walk, your silhouette against the sky, under the crescent moon.

Your love, like a Swift's ballad, a balm to my soul,
In your arms, I found a haven, making me whole.
With each shared sunset, like a Swift's love story, ours unfolds,
In tender verses and choruses, as our love, like the night, takes hold.

So here's to us, to our love, as constant as the tide,
In every sunset we share, in every stride.
Let our story be sung, like a Taylor Swift melody,
An anthem of our love, echoing into 